In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np; np.random.seed(0)
import seaborn as sns

In [2]:
df = pd.read_csv("covid19_tweets.csv")
newdf = df.copy()
newdf = newdf.drop(['user_name','user_location','user_description','user_created','user_verified','date','text','hashtags','source','is_retweet'], axis=1)

In [4]:
import re
def clean_1(s):
    s = s.replace('ðŸž', "")
    s = s.replace('ðŸ¤·â€â™', "")
    s = s.replace("ðŸ”´", "")
    s = s.replace("€™", "")
    s = s.replace("ƒ¼", "")
    s = s.replace("â€˜", "")
    s = s.replace("â€™", "")
    s = s.replace("ðŸ˜˜", "")
    s = s.replace("ðŸ”Š", "")
    s = s.replace("&amp;", "&")
    s = s.replace("ðŸ§©", "")
    s = s.replace(":", "")
    s = s.replace(";", "")
    s = s.replace("'", "")
    s = s.replace("|", "")
    s = s.replace("//", "")
    s = s.replace('📍',"")
    s = s.replace('🏫',"")
    s = s.replace('💯',"")
    s = s.replace('🧵',"")
    s = s.replace('🔊',"")
    s = s.replace("●", "")
    s = s.replace("❤️", "")
    s = s.replace("🙃", "")
    s = s.replace("🤔", "")
    s = s.replace("😉", "")
    s = s.replace("⬇️", "")
    s = s.replace("🔴", "")
    s = s.replace("🤷‍", "")
    s = s.replace("🐞", "")
    s = s.replace("🚨", "")
    s = s.replace("🙌", "")
    s = s.replace("👑", "")
    s = s.replace("🔥", "")
    s = s.replace("👇", "")
    s = s.replace("👀", "")
    s = s.replace("/", "")
    s = s.replace("-", "")
    s = s.replace("\n", "")
    s = s.replace("https", "")
    return str(s)

In [5]:
def clean_2(s):
    s = str(s).upper()
    s = s.replace("[", "")
    s = s.replace("]", "")
    s = s.replace("'", "")
    s = s.replace("CORONAVIRUS","COVID19")
    s = s.replace("CORONAVIRUS, COVID19","COVID19")
    s = s.replace("COVID19, COVID19","COVID19")
    
    return str(s).upper()

In [6]:
df["text_clean"] = ''
df["hashtags_clean"] = ''

In [7]:
for i, row in df.iterrows():
    df.at[i, "text_clean"] = clean_1(row.text)
    df.at[i, "hashtags_clean"] = clean_2(row.hashtags)

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')
for i, row in df.iterrows():
    if i % 1000 == 0:
        print(i)
    if(row["text_clean"] and len(str(row["text_clean"])) < 1000000):
        doc = nlp(str(row["text_clean"]))
        adjectives = []
        nouns = []
        verbs = []
        lemmas = []

        for token in doc:
            lemmas.append(token.lemma_)
            if token.pos_ == "ADJ":
                adjectives.append(token.lemma_)
            if token.pos_ == "NOUN" or token.pos_ == "PROPN":
                nouns.append(token.lemma_)
            if token.pos_ == "VERB":
                verbs.append(token.lemma_)
                
        df.at[i, "selftext_lemma"] = " ".join(lemmas)                
        df.at[i, "selftext_nouns"] = " ".join(nouns)
        df.at[i, "selftext_adjectives"] = " ".join(adjectives)
        df.at[i, "selftext_verbs"] = " ".join(verbs)
        df.at[i, "selftext_nav"] = " ".join(nouns+adjectives+verbs)
        df.at[i, "no_tokens"] = len(lemmas)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000


In [ ]:
df['hashtags'].nunique()

In [ ]:
# group by category, count distinct subreddits and posts
cat_df = df.groupby('hashtags_clean') \
           .agg({'user_location': pd.Series.nunique,
                 'user_name': pd.Series.count}) \
           .rename(columns={'user_location': 'num_locations',
                            'user_name': 'num_posts'}) \
           .sort_values('num_posts', ascending=False)
            
# show top 5 records
cat_df.head(5)

In [ ]:
cat_df.describe()

In [ ]:
cat_df[['num_posts']].plot(kind='box', vert=False, figsize=(6, 2));

In [ ]:
cat_df[['num_locations']].plot(kind='bar', figsize=(7,4));

In [ ]:
# create a data frame slice
sub_df = df[df['hashtags_clean']=='COVID19']

# sample cleaned text and tokens tagged as nouns
sub_df[['text_clean', 'selftext_nouns']].sample(2)

In [ ]:
def my_tokenizer(text):
    return text.split() if text != None else []

In [ ]:
tokens = sub_df.selftext_nouns.map(my_tokenizer).sum()

In [ ]:
from collections import Counter

counter = Counter(tokens)
counter.most_common(20)

In [ ]:
from spacy.lang.en.stop_words import STOP_WORDS

def remove_stopwords(tokens):
    """Remove stopwords from a list of tokens."""
    return [t for t in tokens if t not in STOP_WORDS]

# rebuild counter
counter = Counter(remove_stopwords(tokens))

In [ ]:

# convert list of tuples into data frame
freq_df = pd.DataFrame.from_records(counter.most_common(20),
                                    columns=['token', 'count'])

# create bar plot
freq_df.plot(kind='bar', x='token');

In [ ]:
from wordcloud import WordCloud

def wordcloud(counter):
    wc = WordCloud(width=1200, height=800, 
                   background_color="white", 
                   max_words=200) 
    wc.generate_from_frequencies(counter)

    # Plot
    fig=plt.figure(figsize=(6, 4))
    plt.imshow(wc, interpolation='bilinear')
    plt.axis("off")
    plt.tight_layout(pad=0)
    plt.show()

In [ ]:
wordcloud(counter)

In [ ]:
df['no_tokens'] = df.selftext_lemma\
  .map(lambda l: 0 if l==None else len(l.split()))

In [ ]:
# mean number of tokens by category
df.groupby(['hashtags_clear']) \
  .agg({'no_tokens':'mean'}) \
  .sort_values(by='no_tokens', ascending=False) \
  .plot(kind='bar', figsize=(7,4));

In [ ]:

# render plots as retina or png, because svg is very slow
%config InlineBackend.figure_format = 'retina'

def multi_boxplot(data, x, y, ylim = None):
    '''Wrapper for sns boxplot with cut-off functionality'''
    # plt.figure(figsize=(30, 5))
    fig, ax = plt.subplots()
    plt.xticks(rotation=90) 

    # order boxplots by median
    ordered_values = data.groupby(x)[[y]] \
                         .median() \
                         .sort_values(y, ascending=False) \
                         .index
        
    sns.boxplot(x=x, y=y, data=data, palette='Set2', 
                order=ordered_values)

    fig.set_size_inches(11, 6)
    
    # cut-off y-axis at value ylim
    ax.set_ylim(0, ylim)

In [ ]:
multi_boxplot(df, 'category', 'no_tokens');